In [3]:
# Getting the input data
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
from scipy import stats 
import shutil
from io import StringIO
import statsmodels.formula.api as smf


# Setting the directory

In [4]:
#Setting the directory
os.chdir('/Users/nicolethomas/Google Drive/Grad School/IACMI Project/Python')

#tensile = pd.read_csv("tensile.csv",skiprows = 5,usecols=[2,4],names = ['Axial Extensometer','Axial Force']).dropna(axis = 0) 
#tensile = tensile.apply(pd.to_numeric, errors ='coerce') #Change text format to numeric

#Should add dimensions file here
#Dimensions = pd.read_table("Dimensions.txt")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/nicolethomas/Google Drive/Grad School/IACMI Project/Python'

# Read file & skip lines with heading info

In [5]:
# ... (skipping irrelevant lines of setup code)

def scrub_file(filename, separator=',', skip_header=0):
    # initialize variables
    line = ""
    contents = ""
    lineno = 0
    numfields = 0
    # what conversion are we performing? what does it return if the
    # conversion fails?
    def tofloat(x):
        try:
            return float(x)
        except ValueError:
            return np.nan
    # this is going to handle reading a line from the file, and automatically
    # do anything else we need, such as keep track of which line number
    # we're on.
    def readline():
        nonlocal line, fstream
        #global line
        line = fstream.readline()
        #lineno += 1
        return line
    # set the converter
    convert = tofloat
    # read the file
    with open(filename) as fstream:
        # pass header rows straight through
        for _ in range(skip_header):
            contents.append(readline(fstream))
        # check how many fields fail conversion
        while readline():
            # process the line into fields (comma-separated by default)
            fields = [convert(field) for field in line.split(separator)]
            # how many of these fields are not NaN when an attempt is made
            # to convert them to a float?
            notnan = np.sum(~np.isnan(fields))
            # **This is where all the work is done!** How do you know if a row is
            # bad? Is it bad if all fields fail the conversion? All-but-one? All-but-two?
            # You have to look at the source file and make this determination, then
            # change this inequality. Only rows that pass the test are kept.
            if notnan > 0:
                contents += line
            # update the numfields variable -- this tracks the row with the most
            # number of fields.
            numfields = max(notnan, numfields)
    # now return the good lines (a.k.a. records) as a StringIO object. (StringIO
    # allows you to read from a string object as if it were a file.)
    return StringIO(contents)

# now use the output from scrub_file in your earlier code, e.g.
tensile = pd.read_csv(scrub_file("tensile.csv"),
    usecols=[2,4],
    names = ['Axial Extensometer', 'Axial Force']).dropna(axis = 0)

In [6]:
print('Input_size:')
print(tensile.shape)
print(tensile.head(5))

Input_size:
(1168, 2)
   Axial Extensometer  Axial Force
1           -0.000164    681.18506
2           -0.000153    680.99847
3           -0.000233    679.72717
4           -0.000193    679.57990
5           -0.000287    680.89185


In [ ]:
#Input dimensions and calculate area
b=float(input('Input the width of the sample in millimeters:'))/1000
print (b)
h=float(input('Input the thickness of the sample in millimeters:'))/1000
Area = b*h
print (("Cross-sectional area= %f m^2") %Area)
#Aruvi's area code: Area = int(Dimensions["Width"]*Dimensions["Thickness"])

# Calculate max stress

In [ ]:
#Find the maximum force (in N) and calculste the max stress (in MPa)
Max_load=tensile['Axial Force'].values.max()
Stress = tensile["Axial Force"]/(1000000*Area)

#Read in strain data (already formatted in mm/mm)
Strain = tensile["Axial Extensometer"]

#print(tensile["Stress"].head(5))
print (('The max load on your sample is %f N') %Max_load)
print (('The tensile strength of your sample is %f MPa') %Stress.max()) 

print(Stress.head(5))
print(Strain.head(5))

In [ ]:
#Find split point
def Split_Stress(stress, strain):
    
    # Split_point is strain at whaich standard deviation(ymodulus)/mean(ymodulus) > 10 %
    
    ymodulus = []
    
    for i in range(0,len(strain)-10):
        ymodulus.append((stress[i+10] - stress[i])/(strain[i+10] - strain[i]))
        m = np.mean(ymodulus)
        s = np.std(ymodulus)
        p = (s/m)*100
        if np.floor(p) > 10:
            global Split_Point
            Split_Point = strain[i]  
            break

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []
    strain_deviation = []

    for i in range(0, len(strain)):
        if strain[i] < Split_Point:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])
    
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)

    linear_strain = linear_strain[:,np.newaxis]
           
    E_trial,r,_,_ = np.linalg.lstsq(linear_strain,linear_stress)    # Finding Young's Modulus by forcing to zero intercept
    
    # Calculating mean strain deviation for the linear_stress, linear_strain data
    
    for i in range(0,len(linear_strain)):
        strain_deviation.append((linear_strain[i] - linear_stress[i]/E_trial))
        
    mean_strain_deviation = np.mean(strain_deviation)
    
    # Filtering out the stress, strain data with strain deviation within bounds 
    # of mean strain deviation +/- Extensometer least count
            
    filtered_stress =[]
    
    for i in range(0,len(strain)):
            if mean_strain_deviation-0.00005 < (strain[i]- (stress[i]/E_trial)) < mean_strain_deviation + 0.00005 :
                filtered_stress.append(stress[i])
    break_stress = np.max(filtered_stress)
    return(break_stress)
print('done')

# Finding Mechanical Properties

In [ ]:
# Function to calculate Mechanical Properties

def Mechanical_Properties(stress, strain, split_stress):

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []

    for i in range(0, len(stress)):
        if stress[i] < split_stress:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])

    # Filtering out the linear potion of the ey_strain and stress data to fit a polynomial

    nonlinear_strain =[]
    nonlinear_stress =[]

    for i in range(0,len(stress)):
        if stress[i] > split_stress:
            nonlinear_strain.append(strain[i])
            nonlinear_stress.append(stress[i])
            
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)

    linear_strain = linear_strain[:,np.newaxis]

    E,r,_,_ = np.linalg.lstsq(linear_strain,linear_stress)
    

    #Calculating R^2 value
    K = len(linear_stress)
    r_squared =  1 - r /(K * linear_stress.var())

    #Calculating coefficient of variation
    V = 100*np.sqrt(((1/r_squared)-1)/(K-2))
    
    
    #Fitting a polynomial curve to the nonlinear portion of stress-strain curve
    #p = np.polyfit(nonlinear_strain,nonlinear_stress,20)
    #p1d = np.poly1d(p)
    
    
    #Finding Ultimate Stress and Strain
    ucs = np.amax(stress)
    ucstrain = np.max(strain)
    #oldcode  ucs = np.amax(stress)    ucstrain = np.amax(strain)


    # To find yield stress from initially filtered data of Engineering strain vs Engineering stress
    # This works by finding intersection point of the straight line and polynomial

    x = np.linspace(0.002,0.05,100000)

    for i in range(0,len(x)):
        if (E*(x[i] -0.002)) > (p1d(x[i])):
            yieldstrain = x[i]
            yieldstress = p1d(x[i])
            break 

    #Caculating ductility from filtered data ey_strain
        
    #ductility = (ucstrain - yieldstrain)*100
    return(yieldstress,ucs,E,r_squared,K,V)

print("done")



In [ ]:
#Mechanical properties in different processing conditions

uniaxial_split = Split_Stress(Stress,Strain)   # Passing uniaxial strain, uniaxial stress
print(uniaxial_split)
#mises_split = Split_Stress(Stress, Mises)      # Passing mises strain, uniaxial stress
#print(mises_split)

[y,ucs,E,R2,K,V] = Mechanical_Properties(Stress,Strain,uniaxial_split)


#[y1,ucs1,E1,d1,R2_1,K1,V1] = Mechanical_Properties(Stress,Mises,mises_split)

dir = 'Results'
if not os.path.exists(dir):
    os.makedirs(dir)
else:
    shutil.rmtree(dir)           #removes all the subdirectories!
    os.makedirs(dir)

os.chdir('Results')



Data = pd.DataFrame({'Strain':Strain,'Stress':Stress},columns = ['Strain','Stress'])

Results = pd.DataFrame({'Result Type':['Uniaxial'],'Yield Strength(Mpa)':[y],'Ultimate Strength(Mpa)':[ucs],
                        'Youngs Modulus(Gpa)':[E/1000],'R^2':[R2],'Data Points':[K],'CoV(%)':[V]},
                        columns = ['Result Type','Yield Strength(Mpa)','Ultimate Strength(Mpa)','Youngs Modulus(Gpa)',
                        'R^2','Data Points','CoV(%)'])
#Results = pd.DataFrame({'Result Type':['Uniaxial','Mises'],'Yield Strength(Mpa)':[y,y1],'Ultimate Strength(Mpa)':[ucs,ucs1],
#                        'Youngs Modulus(Gpa)':[E/1000,E1/1000],'Ductility':[d,d1],'R^2':[R2,R2_1],'Data Points':[K,K1],'CoV(%)':[V,V1]},
#                        columns = ['Result Type','Yield Strength(Mpa)','Ultimate Strength(Mpa)','Youngs Modulus(Gpa)',
#                                   'Ductility','R^2','Data Points','CoV(%)'])



Results.to_csv('Results.csv')
Data.to_csv('Data.csv')

print(Results)

# Plot 

In [ ]:
#plt.cla()
#plt.clf()
plt.figure(figsize = (20,15))
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.plot(Strain,Stress,'--')
    #plt.plot(b,a,'-')
    #plt.plot(g,c,'-')
plt.xlabel('Engineering Strain (mm/mm)', fontsize = 20)
plt.ylabel('Engineering Stress (Mpa)', fontsize = 20)
plt.axis([0.0005,0.0025,0,500])
plt.savefig('Eng_Strain_Stress.tiff')
plt.show()




# Linear regression

In [ ]:
# visualize the relationship between the features and the response using scatterplots
#fig, axs = plt.subplots(1, 3, sharey=True)
#tensile.plot(kind='scatter', x=Strain, y=Stress, ax=axs[0], figsize=(16, 8))
#data.plot(kind='scatter', x='Radio', y='Sales', ax=axs[1])
#data.plot(kind='scatter', x='Newspaper', y='Sales', ax=axs[2])

# create a fitted model in one line
#lm = smf.ols(formula='Stress ~ Strain', data=tensile).fit()

# print the coefficients
#lm.params

#fig, ax = plt.subplots()
#fit = np.polyfit(Strain, Stress, deg=1)
#ax.plot(Strain, fit[0] * Strain + fit[1], color='red')
#ax.scatter(Strain, Stress)

#fig.show()

#import seaborn as sns
#sns.lmplot(x=Strain,y=Stress,data=tensile,fit_reg=True) 
# Set the data
#data = pd.DataFrame(tensile)
# Get the linear models
#lm_original = np.polyfit(data.Strain, data.Stress, 1)
 
# calculate the y values based on the co-efficients from the model
#r_x, r_y = (i, x coefficient[0] + coefficient[1])
 
# Put in to a data frame, to keep is all nice
#lm_original_plot = pd.DataFrame({
#Stress : r_x,
#Strain : r_y})


#np.linalg.lstsq(Strain, Stress, rcond=-1)